In [82]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import csv


In [83]:
dhcp_data = []
ctr = 0
with open('data/hashed_mac_dhcp_lease_20170207_20170212.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if ctr > 0:
            dhcp_data.append([ctr]+row[1:])
        ctr += 1

In [84]:
import datetime
import time 

def get_timestamp(s):
    dt = datetime.datetime.strptime(s, "%Y/%m/%d %H:%M:%S")
    return time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
    

In [85]:
dhcp_rdd = sc.parallelize(dhcp_data).map(lambda s: tuple(s[:12]+[get_timestamp(s[12]), get_timestamp(s[13])]+s[14:]))
ip_2_mac = (dhcp_rdd
            .map(lambda s: ((s[9], s[10])))
            .groupByKey()
            .map(lambda s: (s[0], set(s[1])))
           )
mac_2_ip = (dhcp_rdd
            .map(lambda s: ((s[10], s[9])))
            .groupByKey()
            .map(lambda s: (s[0], set(s[1])))
           )

In [86]:
print "Total unique IP addresses:", dhcp_rdd.map(lambda s: s[9]).distinct().count()
print "Total unique MAC addresses:", dhcp_rdd.map(lambda s: s[10]).distinct().count()
print "Total IP addresses used by more than one mac", ip_2_mac.filter(lambda s: len(s[1])>1).count()
print "Total Mac addresses using more than IP addresses", mac_2_ip.filter(lambda s: len(s[1])>1).count()

Total unique IP addresses: 6425
Total unique MAC addresses: 6256
Total IP addresses used by more than one mac 0
Total Mac addresses using more than IP addresses 165


In [100]:
import pickle
with open('data/mac_2_ip_dhcp.pickle', 'w') as f:
    pickle.dump(mac_2_ip.collect(), f)

In [89]:
ip_2_mac_dict = dict((x,y) for x,y in ip_2_mac.collect())

In [102]:
print len(ip_2_mac_dict.keys())

6425


In [98]:
print ip_2_mac_dict['10.9.225.184']

set(['C0:33:5E:28:F:61'])


In [97]:
print mac_2_ip_dict['4C:66:41:F:F4:C7']

set(['10.8.252.6'])


In [101]:
import pickle
with open('data/ip_2_mac_dhcp.pickle', 'w') as f:
    pickle.dump(ip_2_mac.collect(), f)

In [107]:
ip_2_mac_dict = {}
for entry in ip_2_mac.collect():
    ip_2_mac_dict[entry[0]] = entry[1:]

In [108]:
with open('data/ip_2_mac_dict.pickle', 'w') as f:
    pickle.dump(ip_2_mac_dict, f)